# # Regression SHAP MMP

In [2]:
%load_ext autoreload

### Import libraries

In [3]:
import os 
from ML.machine_learning_models import *
from ML.machine_learning_models import Model_Evaluation as ml_evaluation
from ML.data_preprocessing import Dataset
from ML.ml_utils_reg import create_directory, ECFP4, set_global_determinism
from ML.data_utils import *
from tqdm.notebook import tqdm
from regression_shap_mmp.ML.utils_shap import get_df_shap
import joblib
import shap
%autoreload 2

### Parameters

In [4]:
# parameters
# ML models
model_list = ['SVR', 'RFR']
# number of trials
trial_splits = 10
# number of mmp trials
mmp_trials = 1
# fingerprint
fingerprint = 'ECFP4'
# approach
approach = 'regression'

### Load data

In [5]:
# dataset path
db_path = "./dataset/"
main_folder = "./regression_shap_mmp/"
results_path = f'./{main_folder}/{fingerprint}/{approach}/'

In [6]:
df_regression = pd.read_csv(db_path + "chembl_33_pIC50.csv")
df_regression

,smiles,standard_type,pPot,cid,tid
0,Brc1cc2c(NCc3ccccc3)ncnc2s1,IC50,6.617983,CHEMBL3416599,203
1,Brc1cc2c(NCc3ccccn3)ncnc2s1,IC50,5.102153,CHEMBL3416616,203
2,Brc1cc2c(NCc3cccs3)ncnc2s1,IC50,5.862013,CHEMBL3416619,203
3,Brc1cc2c(NCc3ccncc3)ncnc2s1,IC50,5.410833,CHEMBL3416614,203
4,Brc1cc2c(Nc3ccccc3)ncnc2s1,IC50,7.096910,CHEMBL3416621,203
...,...,...,...,...,...
16524,c1csc(-c2n[nH]c3c2Cc2ccccc2-3)c1,IC50,6.031517,CHEMBL212899,279
16525,c1ncc(-c2cc3c(cn2)[nH]c2ncc(-c4ccc(CN5CCCCC5)c...,IC50,6.575118,CHEMBL3582232,220
16526,c1ncc(-c2cc3c(cn2)[nH]c2ncc(-c4ccc(CN5CCCCC5)c...,IC50,6.490797,CHEMBL3582223,220
16527,c1ncc(-c2cc3c(cn2)[nH]c2ncc(-c4ccc(CN5CCCCC5)c...,IC50,6.304518,CHEMBL3582224,220


### Load MMP datasets

In [7]:
mmp_path = "./ccrlib_master/"
df_mmp = pd.read_csv(mmp_path + "df_mmp_final_top10.csv")
df_mmp

,core,as,sub_1,cid_1,sub_2,cid_2,tid,mmp_id,dpot,similarity,mmp_trial
0,COc1cc(Nc2ncnc3cc([*:1])sc23)cc(OC)c1OC,253,c1ccc([*:1])cc1,CHEMBL4541014,Brc1ccc([*:1])cc1,CHEMBL4572443,203,0,1.151490,0.800000,0
1,COc1cc(Nc2ncnc3cc([*:1])sc23)cc(OC)c1OC,253,Ic1ccc([*:1])cc1,CHEMBL4473768,COc1ccc([*:1])cc1,CHEMBL4546122,203,1,0.716003,0.820000,0
2,COc1cc(Nc2ncnc3cc([*:1])sc23)cc(OC)c1OC,253,c1csc([*:1])c1,CHEMBL4460381,Clc1ccc([*:1])cc1,CHEMBL4552482,203,2,2.038223,0.685185,0
3,COc1cc2ncnc(N3CCCc4ccccc43)c2cc1NC(=O)C=CC[*:1],135,CN(C)[*:1],CHEMBL4176787,CC[*:1],CHEMBL4162530,203,3,0.338819,0.814286,0
4,Clc1cc(Nc2ncnc3cccc(O[*:1])c23)ccc1OCc1ccccn1,207,C[*:1],CHEMBL194389,C1CC([*:1])CCO1,CHEMBL193578,203,4,0.736759,0.718750,0
...,...,...,...,...,...,...,...,...,...,...,...
45337,O=c1cc(-c2[nH]c([*:1])nc2-c2ccc(F)cc2)cc[nH]1,3,O=C(O)c1ccc([*:1])cc1,CHEMBL3313935,C#Cc1ccc([*:1])cc1,CHEMBL3314276,260,475,0.504318,0.692308,9
45338,Nc1ccccc1Nc1ccc2c(c1)CCc1ccc(O[*:1])cc1C2=O,73,OCC(O)C[*:1],CHEMBL2152936_CHEMBL2152938,C[*:1],CHEMBL2152777,260,476,1.364568,0.677966,9
45339,O=C1NCc2c(-c3ccccc3Cl)nc(O[*:1])nc2N1c1c(Cl)cc...,144,CN(C)CC[*:1],CHEMBL211426,[*:1],CHEMBL213846,260,477,0.021189,0.616667,9
45340,O=C1c2ccc(Nc3ccccc3)cc2CCc2ccc(O[*:1])cc21,200,C[*:1],CHEMBL2152784,[*:1],CHEMBL2152796,260,478,0.162727,0.673913,9


In [ ]:
df_shapL = []

for target in tqdm(df_mmp.tid.unique()[:]):

    target_path = create_directory(f'./{main_folder}/{fingerprint}/{approach}/{target}/')
    
    print(f"Target: {target}")

    df_regression_target = df_regression[df_regression.tid == target].reset_index(drop=True)
    
    for mmp_trial in range(mmp_trials):
        print(f"MMP trial: {mmp_trial}")
        
        df_mmp_target = df_mmp[(df_mmp.tid == target) & (df_mmp.mmp_trial == mmp_trial)]
        
        for split in ['Random', 'Stratified']:
            print(f"Split: {split}")
            for trial in range(trial_splits):
                print(f"Trial: {trial}")
                
                # train test split
                regression_set, train_idx, test_idx = split_data(df_regression_target, df_mmp_target, test_size=0.5, random_seed=trial, split_type=split)

                # dataset 
                dataset = Dataset(np.array(ECFP4(regression_set.smiles.values)), np.array(regression_set.pPot.values))
                dataset.add_instance("target", regression_set.tid.values)
                dataset.add_instance("smiles", regression_set.smiles.values)
                dataset.add_instance("cid", regression_set.cid.values)
                dataset.add_instance("mmp_id", regression_set.mmp_id.values)
                dataset.add_instance("analog_series_id", regression_set.analog_series_id.values)
                dataset.add_instance("train_test", regression_set.train_test.values)
                dataset.add_instance("similarity", regression_set.similarity.values)
                dataset.add_instance("dPot", regression_set.dpot.values)
                
                # Training set
                training_set = dataset[train_idx]
                # Test set
                test_set = dataset[test_idx]
                
                # Set seed
                set_global_determinism(seed=trial)
                
                for model in model_list:
                    print(f"Model: {model}")
                    # Create saving directory
                    model_fpath = create_directory(f"./regression_mmp/{fingerprint}/{approach}/{target}/{split}/{model}/", verbose=False)
    
                    # ml model
                    model_loaded = None
                    if os.path.exists(os.path.join(model_fpath, f"{model}_{trial}_{mmp_trial}.pkl")):
                        ml_model = joblib.load(os.path.join(model_fpath, f"{model}_{trial}_{mmp_trial}.pkl"))
                        print(f"Model {model}_{trial} loaded")
                        model_loaded = True
                    else:
                        ml_model = MLModel(training_set, model)
                        ml_model = ml_model.model
                        # save model
                        joblib.dump(ml_model, os.path.join(model_fpath, f"{model}_{trial}_{mmp_trial}.pkl"))
    
                    # Model evaluation
                    model_eval_train = ml_evaluation(model=ml_model, data=training_set, model_id=model,
                                                         model_loaded=model_loaded)
                        
                    model_eval_test = ml_evaluation(model=ml_model, data=test_set, model_id=model,
                                                    model_loaded=model_loaded)
                    
                    # Prediction df
                    predictions_train = get_df_results(model_eval_train.predictions, trial=trial, approach=approach, 
                                                        fingerprint=fingerprint, split=split, mm_trial=mmp_trial)

                    # Prediction df
                    predictions_test = get_df_results(model_eval_test.predictions, trial=trial, approach=approach, 
                                                      fingerprint=fingerprint, split=split, mm_trial=mmp_trial)

                    # SHAP
                    if model == 'RFR':
                        model_expl_name = 'TreeExplainer'
                        model_explainer = shap.TreeExplainer(ml_model, feature_perturbation='interventional', data=training_set.features)
                        expected_value = model_explainer.expected_value
                        shap_values_tr = model_explainer.shap_values(X=training_set.features, check_additivity=True).tolist()
                        shap_values_te = model_explainer.shap_values(X=test_set.features, check_additivity=True).tolist()
                        pred_train = ml_model.predict(training_set.features)
                        pred_test = ml_model.predict(test_set.features)

                    elif model == 'SVR':
                        model_expl_name = 'SVETA'
                        model_explainer = ml_model
                        shap_values_tr = [model_explainer.vector_feature_weights(vec) for vec in csr_matrix(training_set.features)]
                        shap_values_te = [model_explainer.vector_feature_weights(vec) for vec in csr_matrix(test_set.features)]
                        expected_value = model_explainer.expected_value[0]
                        pred_train = ml_model.predict(csr_matrix(training_set.features))
                        pred_test = ml_model.predict(csr_matrix(test_set.features))

                    # shap df
                    df_shap_trial_tr = get_df_shap(data=training_set, trial=trial, model=model, split=split, expected_value=expected_value,
                                                   model_explainer=model_expl_name, predictions_test=pred_train, shap_values=shap_values_tr)

                    df_shap_trial_te = get_df_shap(data=test_set, trial=trial, model=model, split=split, expected_value=expected_value,
                                                   model_explainer=model_expl_name, predictions_test=pred_test, shap_values=shap_values_te)
                    
                    df_shap_trial_tr['train_test'] = 'train'
                    df_shap_trial_te['train_test'] = 'test'
                    df_shap_trial_tr['mmp_trial'] = mmp_trial
                    df_shap_trial_te['mmp_trial'] = mmp_trial
                    df_shap_trial = pd.concat([df_shap_trial_tr, df_shap_trial_te])
                    # save df_shap per trial
                    df_shap_trial.to_pickle(os.path.join(target_path, f"df_shap_{model}_{trial}_{mmp_trial}.pkl"))
    
                    df_shapL.append(df_shap_trial)
                    
                    del ml_model, df_shap_trial

df_shap = pd.concat(df_shapL)
df_shap.to_pickle(os.path.join(results_path, f'df_shap.pkl'))
del df_shapL, df_shap